<p><font size="6" color='grey'> <b>
Anwendung Generativer KI
</b></font> </br></p>

<p><font size="5" color='grey'> <b>
Large Language Models und Transformer
</b></font> </br></p>


---

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  Colab-Umfeld</font> </br></p>
# Installierte Python Version
import sys
print(f"Python Version: ",sys.version)
# Installierte LangChain Bibliotheken
print()
print("Installierte LangChain Bibliotheken:")

!pip list | grep '^langchain'
# Unterdrückt die "DeprecationWarning" von LangChain für die Memory-Funktionden
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="langsmith.client")

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  SetUp API-Keys (setup_api_keys)</font> </br></p>

def setup_api_keys():
    """Konfiguriert alle benötigten API-Keys aus Google Colab userdata"""
    from google.colab import userdata
    import os
    from os import environ

    # Dictionary der benötigten API-Keys
    keys = {
        'OPENAI_API_KEY': 'OPENAI_API_KEY',
        'HF_TOKEN': 'HF_TOKEN',
        # Weitere Keys bei Bedarf
    }

    # Keys in Umgebungsvariablen setzen
    for env_var, key_name in keys.items():
        environ[env_var] = userdata.get(key_name)

    return {k: environ[k] for k in keys.keys()}

# Verwendung
all_keys = setup_api_keys()
# Bei Bedarf einzelne Keys direkt zugreifen
# WEATHER_API_KEY = all_keys['WEATHER_API_KEY']

# 1 | Large Language Models
---

LLMs funktionieren im Wesentlichen durch die *Vorhersage des wahrscheinlichsten nächsten Worte*s in einem gegebenen Text, basierend auf einer riesigen Menge an Trainingsdaten aus dem Internet. Dieser Prozess wird durch Billiarden von Parametern gesteuert, die während des Trainings optimiert werden. Ein entscheidender Aspekt ist die Architektur von **Transfomern**, die es LLMs erlaubt, den gesamten Text **parallel** zu verarbeiten, anstatt Wort für Wort, und dabei den Kontext durch einen Mechanismus namens **Self Attention** zu berücksichtigen. Das Ergebnis ist ein System, das erstaunlich flüssige und oft sinnvolle Texte generieren kann, obwohl das genaue *Warum* hinter den Vorhersagen aufgrund der Komplexität und Größe des Modells schwer zu verstehen ist.

# 2 | Foundation Models
---


Die Grundlage großer Sprachmodelle (LLMs) sind **Foundation Models**, die anhand vieler Daten trainiert wurden. Sie können für bestimmte Aufgaben oder Anwendungen angepasst oder optimiert werden. Diese Modelle werden als *Basismodelle* (Foundation Models) bezeichnet, da sie eine grundlegende Ebene an Wissen und Fähigkeiten bieten, auf der spezialisierte Funktionen aufgebaut werden können. Mehrere namhafte Technologieunternehmen und Forschungsorganisationen bieten große Sprachmodelle an. Zu ihnen zählen OpenAI mit Modellen wie **GPT** (Generative Pre-trained Transformer), Google mit **BERT** (Bidirectional Encoder Representations from Transformers) und anderen Varianten sowie Facebook (Meta), das Modelle wie **RoBERTa** (Robustly Optimized BERT Pretraining Approach) anbietet.

Das Trainieren eines großen Sprachmodells von Grund auf erfordert viel Rechenleistung und Fachwissen. Es braucht umfangreiche Daten, die man erst sammeln, bereinigen und verarbeiten muss. Außerdem braucht man leistungsstarke Computer, die so viele Daten verarbeiten können. Das kann sehr teuer werden, für Einzelpersonen und viele Organisationen. Die Entwicklung eines großen Sprachmodells ist sehr anspruchsvoll und dauert oft länger als ein Kurs. In Kursen lernt man, wie man vorhandene Modelle verwendet und optimiert, um bestimmte Probleme zu lösen oder Forschung zu betreiben.



**Im Kurs verwenden wir:**  
- **gpt-4o-mini**: Standardmodell für die meisten Aufgaben.  
- **gpt-4o**: Für detailliertere Antworten.  
- **o3-mini**: Optimiert für logische Schlussfolgerungen.  

<p><font color='black' size="5">
Schrittweiser Modellaufbau
</font></p>


Gerne beschreibe ich das Modell analog zur vorgegebenen Struktur:

Im vorliegenden Code wird ein Chatbot-Modell schrittweise aufgebaut. Der Modellaufbau ist modular und kann als `template` für nachfolgende Konversationsmodelle verwendet werden.

Diese strukturierte Aufteilung macht das Modell robust, wartbar und erweiterbar für verschiedene Anwendungsfälle im Bereich der Konversations-KI.



In [ ]:
# Abschnitt 0: Installation und API-Key
!uv pip install --system --prerelease allow langchain_community langchain_openai

**Erläuterung der externen Funktionen:**

- `from IPython.display import display, Markdown`: Erlaubt die Darstellung von Output in verschiedenen Formaten, einschließlich formatiertem Markdown, in Jupyter Notebooks.
- `from langchain_openai import ChatOpenAI`: Importiert die Klasse `ChatOpenAI`, die zur Integration von OpenAI's Chatbot-Funktionalitäten in die LangChain-Bibliothek genutzt wird.
- `from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder`: Erlaubt die Definition von Vorlagen für Chat-Prompts und Platzhalter für Nachrichten in LangChain.
- `from langchain.schema import HumanMessage, AIMessage, SystemMessage`: Definiert verschiedene Nachrichtentypen in LangChain, die zwischen menschlichen, KI- und Systemnachrichten unterscheiden.
- `from langchain_core.output_parsers.string import StrOutputParser`: Importiert eine Klasse, die zum Parsen von KI-Output in String-Format in der LangChain-Core-Bibliothek verwendet wird.

In [ ]:
# Abschnitt 1: Importe
from IPython.display import display, Markdown

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain_core.output_parsers.string import StrOutputParser

# Abschnitt 2: Konstanten definieren
MODEL = "gpt-4o-mini"
TEMPERATURE = 0.0

DEFAULT_SYSTEM = """
Du bist ein kompetenter KI-Assistent mit breitem Fachwissen.

Deine Antworten sind:
- Klar strukturiert und mit Markdown formatiert
- Praxisorientiert und direkt umsetzbar
- Basierend auf aktuellem Kenntnisstand
- Mit passenden Beispielen versehen
- In verständlicher Sprache formuliert

Bei deiner Arbeit:
- Analysierst du Fragen sorgfältig
- Gibst präzise und relevante Antworten
- Erkennst den Kontext der Anfrage
- Bietest bei Bedarf weiterführende Informationen
- Bleibst sachlich und neutral

Formatiere alle Antworten in Markdown für optimale Lesbarkeit.
"""

# Abschnitt 3: Chat-Komponenten initialisieren
llm = ChatOpenAI(temperature=TEMPERATURE, model=MODEL)

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "{system_prompt}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])

# Abschnitt 4: Funktionen definieren
def format_history(history: list) -> list:
    """Folistiert listKonversationsverlauf für das Prompt-Template."""
    return [
        HumanMessage(content=h.content) if isinstance(h, HumanMessage)
        else AIMessage(content=h.content) if isinstance(h, AIMessage)
        else h
        for h in history
    ]

def process_conversation(question: str, history: list) -> str:
    """Verlistitet eine Konversation und gibt die Antwort zurück."""
    conversation_chain = chat_prompt | llm | StrOutputParser()
    response = conversation_chain.invoke(
        {"system_prompt": DEFAULT_SYSTEM, "history": format_history(history), "question": question}
    )
    return response

def interact_with_ai(question: str, history: list):
    """Führt eine einzelne Interaktion mit der KI durch."""
    print(f"\n🧑‍🦱 Mensch: {question}")
    response = process_conversation(question, history)
    print("\n🤖 KI:", end=" ")
    display(Markdown(response))
    history.extend([HumanMessage(content=question), AIMessage(content=response)])

# Abschnitt 5: Hauptprogramm
history = [SystemMessage(content=DEFAULT_SYSTEM)]
test_questions = [
    "Mein Name ist Ralf",
    "Warum ist der Himmel blau?",
    "Und warum ist er manchmal rot?",
    "Wie ist mein Name?"
]

for question in test_questions:
    interact_with_ai(question, history)

<p><font color='black' size="5">
Neue Konversation starten
</font></p>

In [ ]:
# Abschnitt 5: Hauptprogramm
history = [SystemMessage(content=DEFAULT_SYSTEM)]
question = "Erstelle eine Tabelle der fünf bevölkerungsreichsten Ländern mit Bevölkerung und BIP."
interact_with_ai(question, history)

# 3 | Textgenerierung
---

Die Generierung von Text gehört zu den häufigsten Anwendungsfällen für **Large Language Models (LLMs)**. Die Erstellung von natürlichem Text folgen einem einem ähnlichen Prinzip wie zur Codegenerierung.  

Anstatt eine interaktive Chatumgebung zu verwenden, erfolgt die Textgenerierung durch gezielte **Eingabeaufforderungen (Prompts)**, die an **LangChain** gesendet werden. Das Modell verarbeitet diese Anfragen und liefert darauf basierend den generierten Text.  

Der folgende Code zeigt, wie ein LLM zur Generierung von Text abgefragt werden kann.

<p><font color='black' size="5">
Muster zur Textgenerierung
</font></p>

Bei der grundlegenden Textgenerierung gibt es verschiedene **Eingabeaufforderungsmuster (Prompting-Methoden)**, die sich je nach Detailgrad der bereitgestellten Informationen unterscheiden. Diese Muster beeinflussen, wie das **Large Language Model (LLM)** den gewünschten Text erzeugt.  

Im Folgenden werden verschiedene dieser Muster untersucht und analysiert, wie sie sich auf die Qualität und Struktur der generierten Inhalte auswirken.

* Zero-Shot-Prompt
* One-Shot-Prompt
* Few-Shot-Prompt

<p><font color='black' size="5">
Zero-Shot Textgenerierung
</font></p>

Ein Zero-Shot-Prompt zur Texterstellung ist eine Technik, bei der ein Sprachmodell ohne vorherige Anpassung oder spezielles Training auf eine bestimmte Aufgabe direkt mit einer einzigen Eingabeaufforderung genutzt wird. Um diesen Ansatz erfolgreich anzuwenden, ist es entscheidend, einen präzisen und ausführlichen Prompt zu formulieren, der genau beschreibt, welche Art von Inhalt generiert werden soll. Dabei sollten Stil, Struktur und relevante Details oder Einschränkungen klar angegeben werden. Beispielsweise kann für eine geschäftliche E-Mail der gewünschte Tonfall (formell oder informell), die wesentlichen Inhalte (wie Termin, Thema, Teilnehmer) sowie eine Handlungsaufforderung spezifiziert werden. Da das Modell allein auf die im Prompt enthaltenen Informationen angewiesen ist, sollte das gewünschte Ergebnis möglichst eindeutig formuliert sein. Diese Methode ist flexibel einsetzbar und erlaubt die Erstellung vielfältiger Textarten, ohne dass eine vorherige Modellanpassung erforderlich ist.  

Das folgende Beispiel demonstriert eine Zero-Shot-Eingabeaufforderung: Es werden verschiedene Anforderungen gestellt und Informationen zu einem Studierenden bereitgestellt, jedoch ohne eine explizite Vorlage für das Sprachmodell vorzugeben.

In [ ]:
# Abschnitt 1: Importe
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import SystemMessage, HumanMessage, AIMessage

# Abschnitt 2: Konstanten definieren
MODEL = "gpt-4o-mini"
TEMPERATURE = 0.2
DEFAULT_SYSTEM = """
Du bist ein hilfreicher KI-Assistent, der bei der Erstellung von Empfehlungsschreiben hilft.
"""

# Abschnitt 3: Chat-Komponenten initialisieren
llm = ChatOpenAI(temperature=TEMPERATURE, model=MODEL)
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "{system_prompt}"),
    MessagesPlaceholder("messages"),
    ("human", "{human_input}")
])

# Abschnitt 4: Funktionen definieren
def format_history(history: list[dict]) -> list[dict]:
    """Formatiert den Konversationsverlauf für die Anzeige."""
    formatted_history = []
    for message in history:
        if isinstance(message, HumanMessage):
            formatted_history.append({"role": "user", "content": message.content})
        elif isinstance(message, AIMessage):
            formatted_history.append({"role": "assistant", "content": message.content})
    return formatted_history

def process_conversation(question: str, history: list, llm: ChatOpenAI, chat_prompt: ChatPromptTemplate) -> str:
    """Verarbeitet eine einzelne Konversation."""
    messages = chat_prompt.format_messages(
        system_prompt=DEFAULT_SYSTEM,
        messages=history,
        human_input=question
    )
    response = llm.invoke(messages)  # Verwendung von invoke statt __call__
    history.append(HumanMessage(content=question))
    history.append(response)
    return response.content

def interact_with_ai(question: str, history: list):
    """Führt eine einzelne Interaktion mit der KI durch."""
    response = process_conversation(question, history, llm, chat_prompt)
    print(response)

In [ ]:
# Abschnitt 5: Hauptprogramm
history = [SystemMessage(content=DEFAULT_SYSTEM)]
question = [
    """
    Erstellen Sie ein positives Empfehlungsschreiben für Maria Weber, eine meiner Studierenden im Kurs ML 401 an der Technischen Universität München. Mein Name ist Dr. Julia Berger. Sie bewirbt sich für einen Master of Science in Data Science. Geben Sie mir nur den Haupttext des Schreibens, keine Kopf- oder Fußzeile. Formatieren Sie in Markdown.

    Unten ist ihre Anfrage:

    Sehr geehrte Frau Dr. Berger,

    ich hoffe, es geht Ihnen gut! Ich bin Maria Weber (Matrikelnr.: 7890), eine Absolventin der TU München, die im Juni 2023 meinen Bachelor in Wirtschaftsmathematik abgeschlossen habe.

    Im Wintersemester 2022/23 hatte ich das Privileg, Ihren Kurs ML 401: Grundlagen des maschinellen Lernens zu besuchen. Der Kurs war ein Wahlpflichtfach in meinem Bachelorprogramm. Die Inhalte haben mich sehr begeistert und ich habe mich durchgehend intensiv eingebracht, was sich in der Note 1,0 widerspiegelte.

    Nach meinem Abschluss mit einer Gesamtnote von 1,1 – der besten Note meines Jahrgangs – arbeite ich als Data Analytics Consultant bei der Firma DataTech GmbH. Meine Hauptaufgabe liegt in der Entwicklung von KI-gestützten Analysetools für Geschäftsprozesse. Um meine Expertise im Bereich Data Science zu vertiefen, möchte ich berufsbegleitend einen Master in Data Science absolvieren. Ich schätze Ihre Forschungsarbeit sehr (ich verfolge regelmäßig Ihre Publikationen und habe sie auch in meinem Team vorgestellt) und Ihre Einschätzung wäre für meine Bewerbung sehr wertvoll.

    Ich bewerbe mich für folgende Programme:
    - ETH Zürich, Master of Science in Data Science
    - TU Berlin, Master of Science in Data Engineering

    Dürfte ich Sie um ein Empfehlungsschreiben für diese Bewerbungen bitten? Meinen Lebenslauf habe ich beigefügt und ich stelle Ihnen gerne weitere Informationen zur Verfügung.

    Vielen Dank für die Berücksichtigung meiner Anfrage.

    Mit freundlichen Grüßen
    Maria Weber
    """
]
interact_with_ai(question, history)

<p><font color='black' size="5">
One-Shot Textgenerierung
</font></p>

Ein One-Shot-Prompt zur Textgenerierung ist eine Methode, bei der ein Sprachmodell mit einer einzigen, detaillierten Eingabe angewiesen wird, auf deren Basis es einen zusammenhängenden Text erstellt. Um diese Technik wirkungsvoll einzusetzen, sollte der Prompt klar und präzise formuliert sein und alle relevanten Informationen sowie den gewünschten Kontext enthalten. Dabei ist es ratsam, Stil, Ton und spezifische inhaltliche Aspekte genau zu definieren.  

Möchte man beispielsweise eine atmosphärische Beschreibung einer Küstenstadt generieren lassen, sollten wesentliche Merkmale wie Tageszeit, Stimmung und bestimmte visuelle oder emotionale Eindrücke explizit benannt werden. Eine solche detaillierte Vorgabe erleichtert es dem Modell, die Anforderungen zu erfassen und passgenaue Inhalte zu erstellen.  

Nachdem der Prompt formuliert wurde, kann er direkt in das Textgenerierungstool eingegeben werden. Falls das Ergebnis nicht vollständig den Erwartungen entspricht, lässt sich der Prompt gezielt anpassen, um die Qualität und Relevanz der Ausgabe weiter zu optimieren.

In [ ]:
# Abschnitt 5: Hauptprogramm
history = [SystemMessage(content=DEFAULT_SYSTEM)]
question = """
Basierend auf diesem Beispiel, erstelle ein neues akademisches Empfehlungsschreiben.
Behalte den professionellen Ton bei, aber passe die Details an:

BEISPIEL:
"Ich freue mich sehr, Maria Weber für den Master of Science in Data Science zu empfehlen. Als ihre Dozentin im Kurs ML 401: Grundlagen des maschinellen Lernens an der TU München konnte ich ihre außergewöhnlichen akademischen Fähigkeiten beobachten. Frau Weber schloss den anspruchsvollen Kurs mit der Note 1,0 ab und demonstrierte dabei ein tiefgreifendes Verständnis für maschinelles Lernen und dessen praktische Anwendungen.

Besonders beeindruckend war ihre Fähigkeit, komplexe Konzepte nicht nur zu verstehen, sondern auch auf reale Problemstellungen anzuwenden. In ihrer Position als Data Analytics Consultant bei DataTech GmbH setzt sie diese Fähigkeiten erfolgreich in der Entwicklung KI-gestützter Analysetools ein.

Mit ihrem Bachelorabschluss in Wirtschaftsmathematik (Note 1,1) als Jahrgangsbeste hat Frau Weber bereits bewiesen, dass sie höchsten akademischen Anforderungen gerecht wird. Ihre analytischen Fähigkeiten, gepaart mit ihrer Lernbereitschaft und ihrem Engagement, machen sie zu einer idealen Kandidatin für ein weiterführendes Studium im Bereich Data Science.

Aufgrund ihrer bisherigen Leistungen und ihres Potenzials empfehle ich Frau Weber nachdrücklich für den Master of Science in Data Science. Sie wird zweifellos einen wertvollen Beitrag zum Programm leisten."

ANWEISUNGEN:
1. Erstelle ein neues Empfehlungsschreiben mit ähnlicher Struktur
2. Ändere:
- Namen
- Studienfach
- Universitäten
- Kurse
- Noten
- aktuelle Berufstätigkeit
3. Behalte bei:
- Vier-Absatz-Struktur
- Professionellen Ton
- Spezifische Beispiele für Leistungen
- Klare Empfehlung am Ende

GEWÜNSCHTES FORMAT:
- Nur Haupttext
- Keine Anrede/Grußformel
- In Markdown formatiert
"""

interact_with_ai(question, history)

<p><font color='black' size="5">
Few-Shot Textgenerierung
</font></p>

Ein Few-Shot-Prompt gibt einem Modell eine begrenzte Anzahl von Beispielen, um dessen Reaktion gezielt zu steuern. Diese Methode eignet sich besonders für Sprach- oder Bildgenerierungsmodelle, da sie dem Modell hilft, Muster oder Stile anhand weniger Eingaben zu erkennen und entsprechend nachzubilden.  

Ein typischer Few-Shot-Prompt für ein Textgenerierungsmodell enthält mehrere Beispielpaare aus Eingaben und den gewünschten Ausgaben. Dadurch erhält das Modell eine Orientierung und kann ähnliche Inhalte mit höherer Präzision erzeugen. Dieser Ansatz ermöglicht eine gezielte Anpassung der Modellantworten, ohne dass eine umfassende Trainingsphase erforderlich ist.  

Few-Shot-Prompting verbessert die Qualität und Konsistenz der generierten Texte und macht das Modell flexibler einsetzbar – insbesondere für Aufgaben, die kreative Feinabstimmung oder spezifische Stilvorgaben erfordern.

In [ ]:
# Abschnitt 5: Hauptprogramm
history = [SystemMessage(content=DEFAULT_SYSTEM)]
question = """
    Basierend auf diesen Beispielen, erstelle ein neues akademisches Empfehlungsschreiben.
    Behalte den professionellen Ton bei, aber passe die Details an:

    BEISPIEL 1:
    "Ich freue mich sehr, Maria Weber für den Master of Science in Data Science zu empfehlen. Als ihre Dozentin im Kurs ML 401: Grundlagen des maschinellen Lernens an der TU München konnte ich ihre außergewöhnlichen akademischen Fähigkeiten beobachten. Frau Weber schloss den anspruchsvollen Kurs mit der Note 1,0 ab und demonstrierte dabei ein tiefgreifendes Verständnis für maschinelles Lernen und dessen praktische Anwendungen.

    Besonders beeindruckend war ihre Fähigkeit, komplexe Konzepte nicht nur zu verstehen, sondern auch auf reale Problemstellungen anzuwenden. In ihrer Position als Data Analytics Consultant bei DataTech GmbH setzt sie diese Fähigkeiten erfolgreich in der Entwicklung KI-gestützter Analysetools ein.

    Mit ihrem Bachelorabschluss in Wirtschaftsmathematik (Note 1,1) als Jahrgangsbeste hat Frau Weber bereits bewiesen, dass sie höchsten akademischen Anforderungen gerecht wird. Ihre analytischen Fähigkeiten, gepaart mit ihrer Lernbereitschaft und ihrem Engagement, machen sie zu einer idealen Kandidatin für ein weiterführendes Studium im Bereich Data Science.

    Aufgrund ihrer bisherigen Leistungen und ihres Potenzials empfehle ich Frau Weber nachdrücklich für den Master of Science in Data Science. Sie wird zweifellos einen wertvollen Beitrag zum Programm leisten."

    BEISPIEL 2:
    "Mit großer Überzeugung empfehle ich Thomas Bauer für den Master of Science in Robotik. Als sein Betreuer im Kurs ROB 301: Einführung in die Robotik am Karlsruher Institut für Technologie habe ich sein außerordentliches technisches Talent kennengelernt. Herr Bauer erreichte im Kurs die Note 1,3 und zeigte dabei besondere Stärken in der praktischen Roboterprogrammierung.

    Seine innovative Herangehensweise an komplexe Problemstellungen war bemerkenswert. Als Werkstudent bei RoboTech entwickelte er bereits erfolgreich autonome Navigationssysteme für Industrieroboter und bewies damit seine Fähigkeit, theoretisches Wissen praktisch umzusetzen.

    Sein Bachelorabschluss in Mechatronik (Note 1,4) unterstreicht seine solide technische Grundausbildung. Herr Bauer zeichnet sich durch systematisches Denken, Kreativität bei der Lösungsfindung und ausgeprägte Teamfähigkeit aus - Eigenschaften, die für das angestrebte Masterstudium essentiell sind.

    Ich empfehle Herrn Bauer mit Nachdruck für den Master of Science in Robotik. Seine Motivation und sein technisches Verständnis werden ihn zu einem wertvollen Mitglied des Programms machen."

    BEISPIEL 3:
    "Es ist mir eine Freude, Lisa Schmidt für den Master of Science in Künstlicher Intelligenz zu empfehlen. Während ihres Besuchs meines Kurses AI 501: Deep Learning an der RWTH Aachen bewies sie außergewöhnliche Fähigkeiten im Bereich der künstlichen Intelligenz. Frau Schmidt erzielte die Note 1,2 und entwickelte dabei innovative Lösungsansätze für komplexe Deep-Learning-Aufgaben.

    In ihrer Rolle als KI-Entwicklerin bei AI Solutions hat sie ihr Talent bereits unter Beweis gestellt. Ihre selbstständig entwickelten Neural-Network-Architekturen für Bilderkennungssysteme zeigen ihre Fähigkeit, theoretische Konzepte in praktische Anwendungen zu überführen.

    Mit einem Bachelorabschluss in Informatik (Note 1,3) verfügt Frau Schmidt über eine exzellente Grundlage für weiterführende Studien. Ihre Kombination aus technischem Verständnis, Kreativität und analytischem Denken macht sie zu einer vielversprechenden Kandidatin für ein anspruchsvolles Masterprogramm.

    Ich empfehle Frau Schmidt uneingeschränkt für den Master of Science in Künstlicher Intelligenz. Ihre bisherigen Leistungen und ihr Enthusiasmus für das Fachgebiet lassen keinen Zweifel an ihrem zukünftigen Erfolg."

    ANWEISUNGEN:
    1. Erstelle ein neues Empfehlungsschreiben mit ähnlicher Struktur
    2. Ändere:
        - Namen
        - Studienfach
        - Universitäten
        - Kurse
        - Noten
        - aktuelle Berufstätigkeit
    3. Behalte bei:
        - Vier-Absatz-Struktur
        - Professionellen Ton
        - Spezifische Beispiele für Leistungen
        - Klare Empfehlung am Ende

    GEWÜNSCHTES FORMAT:
    - Nur Haupttext
    - Keine Anrede/Grußformel
    - In Markdown formatiert

    MUSTER FÜR ABSÄTZE:
    1. Einleitung mit Empfehlung und Kontext
    2. Spezifische Leistungen und praktische Erfahrung
    3. Akademischer Hintergrund und persönliche Eigenschaften
    4. Abschließende Empfehlung und Zukunftsprognose
    """

interact_with_ai(question, history)

<p><font color='black' size="5">
Synthetische Daten generieren
</font></p>

LLMs (Large Language Models) eignen sich zur Erzeugung synthetischer Daten, was besonders nützlich für Testszenarien ist, die realistische Informationen oder eine breite demografische Vielfalt erfordern. Diese Modelle können beispielsweise detaillierte Biografien für verschiedene Berufsgruppen generieren, wodurch realitätsnahe Daten für Simulationen, die Entwicklung von Testalgorithmen oder das Training anderer KI-Systeme bereitgestellt werden.  

Ein möglicher Anwendungsfall wäre die Erstellung synthetischer Biografien für Berufsgruppen wie Softwareentwickler, Kinderkrankenschwestern, Finanzanalysten, Naturwissenschaftslehrer an Universitäten oder Marketingmanager. Jede dieser Biografien könnte individuelle Karrierewege, Qualifikationen und berufliche Erfahrungen umfassen, um vielseitige und praxisnahe Tests zu ermöglichen. Auf diese Weise tragen synthetische Daten zur Verbesserung der Systemleistung bei, ohne auf echte personenbezogene Informationen zurückzugreifen.

In [ ]:
# Abschnitt 1: Importe
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import SystemMessage, HumanMessage, AIMessage

# Abschnitt 2: Konstanten definieren
MODEL = "gpt-4o-mini"
TEMPERATURE = 0.2
DEFAULT_SYSTEM = """
Du bist ein Assistent, der synthetische Daten für eine Person in dem von Ihnen angegebenen Berufsfeld generiert. Erstelle eine kurze Biografie der Person, die nicht länger als 5 Sätze ist. Erwähne die Berufsbezeichnung nicht ausdrücklich.
"""

# Abschnitt 3: Chat-Komponenten initialisieren
llm = ChatOpenAI(temperature=TEMPERATURE, model=MODEL)
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "{system_prompt}"),
    MessagesPlaceholder("messages"),
    ("human", "{human_input}")
])

# Abschnitt 4: Funktionen definieren
def format_history(history: list[dict]) -> list[dict]:
    """Formatiert den Konversationsverlauf für die Anzeige."""
    formatted_history = []
    for message in history:
        if isinstance(message, HumanMessage):
            formatted_history.append({"role": "user", "content": message.content})
        elif isinstance(message, AIMessage):
            formatted_history.append({"role": "assistant", "content": message.content})
    return formatted_history

def process_conversation(question: str, history: list, llm: ChatOpenAI, chat_prompt: ChatPromptTemplate) -> str:
    """Verarbeitet eine einzelne Konversation."""
    messages = chat_prompt.format_messages(
        system_prompt=DEFAULT_SYSTEM,
        messages=history,
        human_input=question
    )
    response = llm.invoke(messages)
    history.append(HumanMessage(content=question))
    history.append(response)
    return response.content

def interact_with_ai(question: str, history: list):
    """Führt eine einzelne Interaktion mit der KI durch."""
    response = process_conversation(question, history, llm, chat_prompt)
    print(response)

# Abschnitt 5: Hauptprogramm
history = [SystemMessage(content=DEFAULT_SYSTEM)]
question = ["Softwareentwickler"]

interact_with_ai(question, history)

Nun wird eine CSV-Datei mit mehreren zufällig generierten Biografien erstellt.

In [ ]:
CAREER = [
"Luftballon-Verkäufer",
"Kinderkrankenschwester",
"Talkmaster",
"Hochschullehrer Wirtschaftswissenschaften",
"Zirkus-Zauberer"
]

In [ ]:
from tqdm import tqdm

# Verwenden Sie tqdm, um den Fortschrittsbalken anzuzeigen
history = [SystemMessage(content=DEFAULT_SYSTEM)]
for i in tqdm(range(5), desc="Generating Careers"):
    history = [SystemMessage(content=DEFAULT_SYSTEM)]
    career_choice = CAREER[i]
    result = interact_with_ai(career_choice, history)
    print(result)

# 4 | Textzusammenfassung
---

Große Sprachmodelle (LLMs) wie GPT-4 fassen Texte zusammen, indem sie zentrale Inhalte extrahieren und kompakt wiedergeben. Sie erfassen den Kontext sowie semantische Strukturen und erstellen eine prägnante Version, die die Hauptaussagen erhält. Durch ihre Fähigkeit, verschiedene Textarten zu analysieren – von Fachartikeln bis hin zu narrativen Texten - liefern sie verständliche und relevante Zusammenfassungen. Ihre Flexibilität in Bezug auf Länge und inhaltliche Schwerpunkte macht sie zu einem effizienten Werkzeug für die schnelle Informationsverarbeitung.

<p><font color='black' size="5">
Einzelnes PDF zusammenfassen
</font></p>

Zunächst wird die Zusammenfassung einer einzelnen PDF-Datei betrachtet. LangChang nutzt spezielle Dokumentlader, um verschiedene Dateiformate wie PDFs zu verarbeiten. Für unterschiedliche Datentypen stehen spezifische Lader zur Verfügung. Im folgenden Code wird eine PDF-Datei geladen und mithilfe einer allgemeinen Systemaufforderung analysiert und zusammengefasst.

In [ ]:
# Abschnitt 0: Installation und API-Key
!uv pip install --system --prerelease allow langchain-community langchain_openai pypdf

Der folgende Code demonstriert die Nutzung der Funktion **„load_summarize_chain“**, um mithilfe eines Large Language Models (LLM) und des Kettentyps **„map_reduce“** eine Zusammenfassung zu erstellen. Zunächst wird das PDF-Dokument *Attention Is All You Need* über den **„PyPDFLoader“** von der angegebenen URL (**„https://arxiv.org/pdf/1706.03762“**) geladen. Anschließend wird der Inhalt durch **„load_and_split“** in kleinere Abschnitte unterteilt. Diese Segmente werden dann der Verarbeitungskette (**„chain.run(docs)“**) übergeben, die den Text verarbeitet und verdichtet. Schließlich erfolgt die Ausgabe des zusammengefassten Inhalts im **Markdown-Format**, um die ursprüngliche Formatierung beizubehalten.

In [ ]:
# Abschnitt 1: Importe
from IPython.display import display, Markdown

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain_core.output_parsers.string import StrOutputParser

# Importe für einzelne Zusammenfassung
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain import PromptTemplate

# Importe für mehrere Zusammenfassungen
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema.document import Document

# Abschnitt 2: Konstanten definieren
MODEL = "gpt-4o-mini"  # Aktuellstes GPT-4 Modell
TEMPERATURE = 0.0

TEMPLATE = """
Schreiben Sie eine kurze Zusammenfassung der präsentierten Informationen. Schreiben Sie die Zusammenfassung auf Deutsch.

{text}

ZUSAMMENFASSUNG:"""

# Abschnitt 3: Chat-Komponenten initialisieren
llm = ChatOpenAI(temperature=TEMPERATURE, model=MODEL)

# Abschnitt 4: Funktionen definieren

def load_document(url: str):
    """Lädt ein Dokument von einer URL und splittet es in Chunks"""
    loader = PyPDFLoader(url)
    return loader.load_and_split()

def summarize_document(docs, chain):
    """Fasst ein Dokument zusammen"""
    # Dokumente zu einem String zusammenfügen
    text = " ".join([doc.page_content for doc in docs])
    return chain.invoke({"text": text})

# Create prompt template using LCEL
prompt = PromptTemplate.from_template(TEMPLATE)

# Initialize chain with custom prompt and LCEL
chain = prompt | llm | StrOutputParser()

# Abschnitt 5: Hauptprogramm
url = "https://arxiv.org/pdf/1706.03762"
docs = load_document(url)
summary = summarize_document(docs, chain)
display(Markdown("## ✨ Zusammenfassung:"))
display(Markdown(summary))

<p><font color='black' size="5">
Mehrere PDFs zusammenfassen
</font></p>

Diese vier wissenschaftlichen Arbeiten haben wesentliche Fortschritte im Bereich der generativen KI und der Verarbeitung natürlicher Sprache (NLP) ermöglicht.  

1. **"Attention Is All You Need" (Vaswani et al., 2017)** führte die **Transformer-Architektur** ein, die sich als grundlegendes Modell für moderne NLP-Systeme etabliert hat. Der Verzicht auf rekurrente Strukturen und die Einführung des Selbstaufmerksamkeitsmechanismus ermöglichten effizientere und leistungsfähigere Modelle wie GPT und BERT.  

2. **"BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding" (Devlin et al., 2018)** revolutionierte die Kontextverarbeitung in NLP-Modellen durch **bidirektionales Training**, das es ermöglichte, sowohl vorhergehende als auch nachfolgende Wörter gleichzeitig zu analysieren. Dadurch erzielte BERT erhebliche Verbesserungen bei zahlreichen NLP-Aufgaben.  

3. **"Language Models are Few-Shot Learners" (Brown et al., 2020)** untersuchte das Potenzial **großskalierter Transformer-Modelle** am Beispiel von GPT-3. Die Studie zeigte, dass die Leistung solcher Modelle durch Skalierung verbessert wird und sie auch ohne aufgabenspezifisches Training effektive Ergebnisse liefern können.  

4. **"Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer" (Raffel et al., 2019)** präsentierte **T5 (Text-to-Text Transfer Transformer)**, ein Modell, das alle NLP-Aufgaben in ein einheitliches Text-zu-Text-Format überführt. Dadurch wurde das Transferlernen zwischen verschiedenen Aufgaben vereinfacht.  

Diese Arbeiten bilden zusammen die Grundlage für viele moderne NLP-Modelle und deren Anwendungen in generativer KI.

In [ ]:
# Abschnitt 5: Hauptprogramm - LCEL-Version
urls = [
    "https://arxiv.org/pdf/1706.03762",
    "https://arxiv.org/pdf/1810.04805",
    "https://arxiv.org/pdf/2005.14165",
    "https://arxiv.org/pdf/1910.10683"
]

summaries = []

for url in urls:
    print(f"Verarbeite Artikel: {url}")
    docs = load_document(url)                       # Dokument laden
    summary_part = summarize_document(docs, chain)  # Zusammenfassung erstellen
    display(Markdown("## ✨ Zusammenfassung:"))
    display(Markdown(summary_part))                 # Zusammenfassung ausgeben
    summaries.append(summary_part)                  # Zusammenfassungen sammeln

Nachdem die einzelnen Artikel zusammengefasst wurden, folgt die Erstellung einer umfassenden Übersicht durch die Kombination dieser Zusammenfassungen. Hierfür werden die Texte zunächst zu einer langen Zeichenfolge zusammengeführt.  

Das Endergebnis wird im **Markdown-Format** ausgegeben, um eine klare Struktur und Lesbarkeit zu gewährleisten. Auf diese Weise entsteht eine prägnante, aber inhaltlich umfassende Synthese der Originalartikel.

In [ ]:
# Gesamtzusammenfassung erstellen
gesamt_text = " ".join(summaries)
gesamt_zusammenfassung = chain.invoke({"text": gesamt_text})

display(Markdown("## ✨ Gesamt-Zusammenfassung:"))
display(Markdown(gesamt_zusammenfassung))

# 5 | Textklassifizierung
---

Große Sprachmodelle (LLMs) haben die Textklassifizierung grundlegend verändert, indem sie einen flexibleren und effizienteren Ansatz als herkömmliche maschinelle Lernmethoden bieten. Während traditionelle Verfahren umfangreiche, manuell annotierte Datensätze benötigten, um Modelle zu trainieren, bieten LLMs eine zeitsparende Alternative.  

Durch die **Few-Shot Klassifizierung**, die mit beschriftete Beispiele funktioniert, können LLMs Texte allein auf Basis weniger natürlichsprachlichen Beispiele kategorisieren. Diese Fähigkeit beruht auf dem umfassenden Wissen, das die Modelle während ihres Trainings mit großen Textkorpora erworben haben.  

Dieser Ansatz reduziert den Aufwand für die Einrichtung einer Textklassifizierung erheblich und ermöglicht eine schnelle Anpassung an neue Anforderungen. Dadurch eignen sich LLMs für verschiedene Anwendungsfälle, darunter **Stimmungsanalyse, Themenkategorisierung, Spam-Erkennung und Inhaltsmoderation**.  

Im folgenden Code werden SMS-Texte analysiert. Anschließend erfolgt deren Klassifizierung mithilfe eines LLMs.

Im folgenden Programm erfolgt eine Klassifizierung der SMS-Texte.


In [ ]:
# Abschnitt 0: Installation und API-Key
!uv pip install --system --prerelease allow langchain-community langchain_openai pypdf

In [ ]:
# Abschnitt 1: Importe
import pandas as pd
import os
from IPython.display import display, Markdown

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain_core.output_parsers.string import StrOutputParser

# Abschnitt 2: Konstanten definieren
MODEL = "gpt-4o-mini"  # Aktuellstes GPT-4 Modell
TEMPERATURE = 0.0

DEFAULT_SYSTEM = """
Du bist ein Experte für die Klassifikation von SMS-Nachrichten.

Deine Aufgabe:
- Klassifiziere SMS-Nachrichten als 'spam' oder 'ham'
- Gib nur das Klassifikationsergebnis zurück
- Verwende 'sonstiges' wenn keine klare Zuordnung möglich ist
- Keine zusätzlichen Erklärungen

Beachte die Beispiele für die Klassifikation:
- spam: Werbung, Gewinnspiele, kostenpflichtige Dienste
- ham: Normale Kommunikation, persönliche Nachrichten
"""

# Abschnitt 3: Chat-Komponenten initialisieren
llm = ChatOpenAI(temperature=TEMPERATURE, model=MODEL)

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "{system_prompt}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])

# Abschnitt 4: Funktionen definieren
def format_history(history: list) -> list:
    """Formatiert den Konversationsverlauf für das Prompt-Template."""
    return [
        HumanMessage(content=h.content) if isinstance(h, HumanMessage)
        else AIMessage(content=h.content) if isinstance(h, AIMessage)
        else h
        for h in history
    ]

def process_conversation(question: str, history: list) -> str:
    """Verarbeitet eine Konversation und gibt die Antwort zurück."""
    conversation_chain = chat_prompt | llm | StrOutputParser()
    response = conversation_chain.invoke(
        {"system_prompt": DEFAULT_SYSTEM, "history": format_history(history), "question": question}
    )
    return response

def create_training_examples() -> str:
    """ Erstellt Trainingsbeispiele aus den Beispiel-SMS """
    examples = []
    for label, messages in train_sms.items():
        for msg in messages:
            examples.append(f"{label}\t{msg}")
    return "\n".join(examples)

def classify(sms_text: str, history: list) -> str:
    """ Klassifiziert eine einzelne SMS-Nachricht """
    # Erstelle Prompt mit Trainingsbeispielen
    full_prompt = f"""
    Klassifiziere die folgende SMS basierend auf diesen Beispielen:

    {create_training_examples()}

    Gebe als Antwort nur "spam" oder "ham" zurück.
    Wenn keine Entscheidung möglich ist dann "sonstiges".
    Gebe keine Begründung.

    Hier ist die SMS:
    {sms_text}
    """
    response = process_conversation(full_prompt, history)
    return response.strip()

# Abschnitt 5: Hauptprogramm
history = [SystemMessage(content=DEFAULT_SYSTEM)]

# Beispiel-SMS für das Training
train_sms = {
    'spam': [
        "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
        "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv",
        "WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only."
    ],
    'ham': [
        "Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",
        "Ok lar... Joking wif u oni...",
        "U dun say so early hor... U c already then say..."
    ]
}

test_sms = [
    "WINNER!! As a valued customer you have won £1000 prize!",
    "Hey, what time is the meeting tomorrow?",
    "Free entry! Win an iPhone now! Click here:",
    "I'll be home in 10 minutes"
]

display(Markdown("## ✨ Klassifiziere Test-SMS:"))
display(Markdown("---"))

# SMS-Klassifizierung
for i, sms in enumerate(test_sms):
    result = classify(sms, history)
    print(f"\nTest SMS #{i+1}:")
    print(f"Text: {sms}")
    print(f"Klassifizierung: {result}")

<p><font color='black' size="5">
Namen klassifizieren
</font></p>

Die Analyse von Wörtern in Texten stellt für künstliche Intelligenz eine interessante Herausforderung dar, besonders bei der Zero-Shot Klassifizierung ohne Trainingsdaten. Hierbei muss das Modell spontan entscheiden, ob ein Wort ein Name oder ein reguläres Wort ist. Im folgenden Text soll getestet werden, ob eine generative KI in der Lage ist, Wörter korrekt in die Kategorien "Name" oder "Nicht-Name" einzuordnen, ohne zuvor mit ähnlichen Beispielen trainiert worden zu sein. Die zu analysierenden Wörter enthalten sowohl gewöhnliche Begriffe als auch verschiedene Arten von Namen wie Vornamen, Nachnamen oder Firmennamen. Die zentrale Frage lautet: Kann das Modell durch sein grundlegendes Sprachverständnis diese Unterscheidung treffen?

Nun wird ein Large Language Model (LLM) definiert, das den Text analysiert.

Letztendlich wird eine einzelne Eingabeaufforderung genutzt, um diese Namen zu identifizieren und zu kategorisieren.

In [ ]:
# Abschnitt 1: Importe
from IPython.display import display, Markdown

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain_core.output_parsers.string import StrOutputParser

# Abschnitt 2: Konstanten definieren
MODEL = "gpt-4o-mini"  # Aktuellstes GPT-4 Modell
TEMPERATURE = 0.0

DEFAULT_SYSTEM = """
Du bist ein präziser Namensextraktor. Deine Aufgabe:
- Extrahiere alle Namen von Menschen, Produkten und Unternehmen aus dem Text
- Gib nur die gefundenen Namen zurück, einer pro Zeile
- Keine zusätzlichen Erklärungen oder Formatierungen
- Keine Duplikate
"""

SAMPLE = """
Anna und Sarah sind seit dem Studium befreundet. Nach ihrem Abschluss hatten sie ein Vorstellungsgespräch bei Frau Weber für eine Stelle bei der Technovision GmbH, deren Hauptprodukt Futurtech heißt und von der Programmiererin Maria Schmidt entwickelt wurde. Nach dem erfolgreichen Gespräch trafen sie sich mit ihren Freunden Rafael und Aida zum Feiern. An ihrem ersten Arbeitstag lernten sie drei weitere Mitarbeiter kennen: Richard, Lisa und Pia. Später kam noch Matthias Müller zum Team dazu.
"""

# Abschnitt 3: Chat-Komponenten initialisieren
llm = ChatOpenAI(temperature=TEMPERATURE, model=MODEL)

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "{system_prompt}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])

# Abschnitt 4: Funktionen definieren
def format_history(history: list) -> list:
    """Formatiert den Konversationsverlauf für das Prompt-Template."""
    return [
        HumanMessage(content=h.content) if isinstance(h, HumanMessage)
        else AIMessage(content=h.content) if isinstance(h, AIMessage)
        else h
        for h in history
    ]

def process_conversation(question: str, history: list) -> str:
    """Verarbeitet eine Konversation und gibt die Antwort zurück."""
    conversation_chain = chat_prompt | llm | StrOutputParser()
    response = conversation_chain.invoke(
        {"system_prompt": DEFAULT_SYSTEM, "history": format_history(history), "question": question}
    )
    return response

def extract_names(text: str, history: list) -> str:
    """ Extrahiert Namen aus einem Text """
    response = process_conversation(text, history)
    return response.strip()

# Abschnitt 5: Hauptprogramm
history = [SystemMessage(content=DEFAULT_SYSTEM)]

# Namen extrahieren und anzeigen
names = extract_names(SAMPLE, history)
display(Markdown("## 📛 Gefundene Namen:"))
display(Markdown(names))

# 6 | LLM schreibt ein Buch
---

Dieser Abschnitt zeigt, wie ein Large Language Model (LLM) beim Schreiben eines Buches genutzt werden kann. Da ein Buch eine komplexe, langfristige Aufgabe darstellt, erfolgt der Prozess schrittweise.  

Zunächst wird ein **Thema** gewählt, auf dessen Basis das LLM einen Titel und eine kurze Zusammenfassung erstellt. Diese Zusammenfassung dient als inhaltlicher Leitfaden. Anschließend wird ein **Inhaltsverzeichnis** generiert, das die Kapitel und Abschnitte strukturiert.  

Jedes **Kapitel** wird in separaten Sitzungen mit dem LLM ausgearbeitet, wobei die zuvor erstellte Gliederung zur Orientierung dient. Dieser iterative Ansatz gewährleistet Konsistenz und ermöglicht eine detaillierte Ausarbeitung.  

Durch eine methodische Vorgehensweise verwandelt sich eine anfängliche Idee in ein gut strukturiertes Buch. Für kreative Ergebnisse wird ein Sprachmodell mit einer Temperatur von 0,7 verwendet.

In [ ]:
!uv pip install --system --prerelease allow pdfkit
!sudo apt-get install -qq -y wkhtmltopdf > /dev/null 2>&1

In [ ]:
# Abschnitt 1: Importe
from IPython.display import display, Markdown
import markdown
import pdfkit

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain_core.output_parsers.string import StrOutputParser

# Abschnitt 2: Konstanten definieren
MODEL = "gpt-4o-mini"  # Aktuellstes GPT-4 Modell
TEMPERATURE = 0.7

DEFAULT_SYSTEM = """
Du bist ein kreativer Autor und Schriftsteller.

Deine Texte sind:
- Fesselnd und gut strukturiert
- Reich an Details und Beschreibungen
- Konsistent im Stil und der Erzählweise
- Mit klarer Handlung und Charakterentwicklung
- In ansprechendem Deutsch verfasst

Bei deinem Schreiben:
- Entwickelst du überzeugende Charaktere
- Erschaffst atmosphärische Szenen
- Behältst den roten Faden bei
- Verwendest kreative Sprachbilder
- Achtest auf Spannung und Dynamik
"""

NUM_CHAPTER = 10
SUBJECT = "Sci-Fi Fantasy Thriller"

# Prompt-Templates
TITLE_PROMPT = ChatPromptTemplate.from_template(
    "Gebe mir einen zufälligen Titel für ein Buch zum Thema {subject}. "
    "Gibt nur den Titel zurück, keinen zusätzlichen Text."
)

SYNOPSIS_PROMPT = ChatPromptTemplate.from_template(
    "Gebe mir eine Inhaltsangabe für ein Buch zum Thema {subject}. "
    "Gebe nur die Inhaltsangabe zurück, keinen zusätzlichen Text."
)

TOC_PROMPT = ChatPromptTemplate.from_template(
    "Gebe mir ein Inhaltsverzeichnis für ein Buch mit dem Titel {title} "
    "für ein Buch zum Thema {subject}. Die Buchzusammenfassung lautet {synopsis}. "
    "Gebe das Inhaltsverzeichnis als liste von Kapitelüberschriften zurück. "
    "Erstelle nicht mehr als {num_chapters} Kapitel. "
    "Trenne Kapitelnummer und Kapitelüberschrift mit einem Pipe-Zeichen '|'. "
    "Gebe nur die Kapitelnamen zurück, keinen zusätzlichen Text."
)

BOOK_PROMPT = ChatPromptTemplate.from_template(
    "Schreibe Kapitel {chapter_num} mit dem Titel {chapter_title} für ein Buch "
    "mit dem Titel {title} zum Thema {subject}. Die Buchzusammenfassung lautet {synopsis}. "
    "Das Inhaltsverzeichnis lautet {toc}. "
    "Gebe nur den Kapiteltext zurück, keine Kapitelüberschrift, keinen Kapiteltitel, "
    "keine Kapitelnummer, keinen zusätzlichen Text."
)

# Abschnitt 3: Chat-Komponenten initialisieren
llm = ChatOpenAI(temperature=TEMPERATURE, model=MODEL)

Einfache Hilfsfunktionen werden implementiert, um das LLM mit einer benutzerdefinierten Systemaufforderung zu befragen. Diese Aufforderung informiert das Modell darüber, dass es beim Schreiben eines Buches assistiert.

In [ ]:
# Abschnitt 4: Funktionen definieren
def process_conversation(prompt: ChatPromptTemplate, **kwargs) -> str:
    """ Verarbeitet eine Konversation mit dem LLM unter Verwendung eines ChatPromptTemplates. """
    formatted_prompt = prompt.format_messages(**kwargs)
    response = llm.invoke(formatted_prompt)
    return response.content.strip(" '\"")

def render_chapter(
    chapter_num: int,
    chapter_title: str,
    title: str,
    subject: str,
    synopsis: str,
    toc: str
) -> str:
    """ Rendert ein Kapitel basierend auf den übergebenen Parametern. """
    txt = process_conversation(
        BOOK_PROMPT,
        chapter_num=chapter_num,
        chapter_title=chapter_title,
        title=title,
        subject=subject,
        synopsis=synopsis,
        toc=toc
    )
    return txt

def markdown_string_zu_pdf(markdown_string, ausgabedatei):
    """ Konvertiert einen Markdown-String in eine PDF-Datei. """
    html = markdown.markdown(markdown_string)
    html = f"""
    <!DOCTYPE html>
    <html lang="de">
    <head>
        <meta charset="UTF-8">
    </head>
    <body>
        {html}
    </body>
    </html>
    """
    try:
        pdfkit.from_string(html, ausgabedatei)
        print(f"PDF erfolgreich erstellt: {ausgabedatei}")
    except Exception as e:
        print(f"Fehler bei der PDF-Erstellung: {e}")


<p><font color='black' size="5">
Titel, Zusammenfassung, Inhaltsverzeichnis
</font></p>

Für dieses Buch kann der Benutzer das Thema über die Variable `SUBJECT` festlegen. Anschließend wird das LLM angewiesen, einen zufälligen Titel basierend auf diesem Thema zu generieren. Dabei wird darauf geachtet, dass die Eingabeaufforderung präzise formuliert ist, um zu vermeiden, dass das LLM zusätzliche Einleitungen wie „Hier ist ein zufälliger Titel“ hinzufügt.

In [ ]:
# Abschnitt 5: Hauptprogramm
title = process_conversation(TITLE_PROMPT, subject=SUBJECT)
print(f"Buchtitel: {title}")

Nachdem der Titel festgelegt wurde, kann nun eine zufällige Zusammenfassung des Buches generiert werden. Diese gibt eine erste inhaltliche Orientierung und dient als Grundlage für die weitere Strukturierung.

In [ ]:
synopsis = process_conversation(SYNOPSIS_PROMPT, subject=SUBJECT)
display(Markdown("## ✨ Zusammenfassung:"))
display(Markdown(synopsis))

Nun wird das Inhaltsverzeichnis erstellt, wobei alle zuvor generierten Informationen berücksichtigt werden. Ein spezifisches Format wird vorgegeben, um eine klare Struktur zu gewährleisten. Obwohl die Kapitelnummern leicht ableitbar wären, werden sie explizit angefordert, da das LLM sie ohnehin bereitstellen möchte. Der Ansatz erleichtert eine konsistente Struktur und ermöglicht es, die Nummerierung später gezielt zu entfernen, falls erforderlich.

In [ ]:
toc = process_conversation(
    TOC_PROMPT,
    title=title,
    subject=SUBJECT,
    synopsis=synopsis,
    num_chapters=NUM_CHAPTER
)
toc = [chapter.strip().split("|") for chapter in toc.split("\n") if chapter.strip()]
display(Markdown("## 🗒️ Inhaltsverzeichnis:"))
for chapter_num, chapter_title in toc:
    display(Markdown(f"{chapter_num}. {chapter_title}"))


<p><font color='black' size="5">
Erstellen der Kapitel des Buches
</font></p>


Nun wird eine Funktion erstellt, die den Text eines Kapitels generiert. Damit das LLM über ausreichend Kontext verfügt, werden die Zusammenfassung, das Inhaltsverzeichnis und die entsprechende Kapitelnummer übergeben.

In [ ]:
book_content = f"# {title}"
for chapter_num, chapter_title in toc:
    print(f"Generiere Kapitel {chapter_num}: {chapter_title}")
    chapter_text = render_chapter(
        chapter_num,
        chapter_title,
        title,
        SUBJECT,
        synopsis,
        toc
    )
    book_content += f"\n## {chapter_title}\n{chapter_text}"

markdown_string_zu_pdf(book_content, "mein_buch.pdf")

Das generierte Buch steht nun zum Download bereit.

In [ ]:
from google.colab import files
files.download("mein_buch.pdf")

# A | Aufgabe
---

Die Aufgabestellungen unten bieten Anregungen, Sie können aber auch gerne eine andere Herausforderung angehen.


<p><font color='black' size="5">
Sentiment-Analyse von Produktbewertungen mit LLM Zero-Shot-Klassifizierung
</font></p>

**Aufgabenstellung**
Entwickeln Sie ein Python-Programm, das mithilfe eines Large Language Models (LLM) Produktbewertungen in die Kategorien "Positiv", "Neutral" oder "Negativ" einordnet und zusätzlich die betroffenen Produktaspekte (Qualität, Preis, Lieferung, Service) identifiziert.

**Lernziele**
Nach Abschluss dieser Übung können Sie:
- Zero-Shot-Klassifizierung für mehrere Kategorien implementieren
- Aspekt-basierte Sentiment-Analyse durchführen
- Komplexe Prompts für Mehrfachklassifizierung erstellen
- Strukturierte Ausgaben aus LLM-Antworten generieren

**Aufgabendetails**

```python
# Beispieldaten
reviews = [
    "Die Qualität des Produkts ist hervorragend, allerdings finde ich den Preis zu hoch.",
    "Schnelle Lieferung, guter Service, faire Preise - besser geht es nicht!",
    "Nach zwei Wochen ging das Gerät kaputt. Der Kundenservice war bei der Reklamation leider keine Hilfe.",
    "Durchschnittliche Qualität, erfüllt seinen Zweck. Lieferung dauerte etwas länger als angegeben."
]
```

**Beispiel-Prompt**
```
Analysiere die folgende Produktbewertung hinsichtlich:
1. Gesamtsentiment (Positiv/Neutral/Negativ)
2. Erwähnte Produktaspekte (Qualität/Preis/Lieferung/Service)
3. Sentiment pro Aspekt

Bewertung: {review}

Antworte im Format:
Gesamtsentiment: [Kategorie]
Identifizierte Aspekte:
- [Aspekt]: [Sentiment]
Begründung: [Analyse]
```